In [9]:
from src.meta_results_r_data import prepare_fforma_data_m3_tourism
from src.meta_learner import MetaLearner
from src.meta_evaluation import calc_errors_widing

from src.utils import FREQ_DICT

In [2]:
X_train_df, preds_train_df, y_insample_train_df, y_train_df, \
    X_test_df, preds_test_df, y_insample_test_df, y_test_df = prepare_fforma_data_m3_tourism('data', kind='TOURISM')

% of missing values, train set
x_acf10               7.551487
diff1_acf1            7.551487
diff1_acf10           7.551487
diff2_acf1            7.551487
diff2_acf10           8.543097
seas_acf1            39.511823
arch_lm               9.687262
garch_acf             0.076278
arch_r2              13.196034
garch_r2             13.348589
nonlinearity          1.144165
x_pacf5               1.144165
diff1x_pacf5          1.754386
diff2x_pacf5          2.059497
seas_pacf            39.511823
e_acf10               7.551487
seasonal_strength    39.511823
peak                 39.511823
trough               39.511823
hw_alpha             39.511823
hw_beta              39.511823
hw_gamma             39.511823
dtype: float64


% of missing values, test set
x_acf10               1.754386
diff1_acf1            1.754386
diff1_acf10           2.059497
diff2_acf1            2.059497
diff2_acf10           2.212052
seas_acf1            39.511823
arch_lm               7.551487
arch_r2               7

In [3]:
df_seasonality = X_train_df[['unique_id']].copy()
df_seasonality['seasonality'] = df_seasonality['unique_id'].apply(lambda x: FREQ_DICT[x[0]])

In [4]:
params = {'n_estimators': 10,
          'eta': 0.5,
          'max_depth': 14,
          'subsample': 0.92,
          'colsample_bytree': 0.77,
          'df_seasonality': df_seasonality}

In [5]:
model = MetaLearner(params).fit(X_train_df, preds_train_df, y_train_df, y_insample_train_df)

Calculating errors...
Training...
[1]	training's FFORMA-loss: 1.04384	valid_1's FFORMA-loss: 1.00339
Training until validation scores don't improve for 10 rounds
[2]	training's FFORMA-loss: 1.00724	valid_1's FFORMA-loss: 0.980701
[3]	training's FFORMA-loss: 0.977392	valid_1's FFORMA-loss: 0.962166
[4]	training's FFORMA-loss: 0.952902	valid_1's FFORMA-loss: 0.947014
[5]	training's FFORMA-loss: 0.932218	valid_1's FFORMA-loss: 0.935089
[6]	training's FFORMA-loss: 0.914383	valid_1's FFORMA-loss: 0.925017
[7]	training's FFORMA-loss: 0.89857	valid_1's FFORMA-loss: 0.916474
[8]	training's FFORMA-loss: 0.884879	valid_1's FFORMA-loss: 0.908917
[9]	training's FFORMA-loss: 0.872703	valid_1's FFORMA-loss: 0.90365
[10]	training's FFORMA-loss: 0.861449	valid_1's FFORMA-loss: 0.89849
Did not meet early stopping. Best iteration is:
[10]	training's FFORMA-loss: 0.861449	valid_1's FFORMA-loss: 0.89849


In [6]:
preds = model.predict(X_test_df, base_model_preds=preds_test_df)

In [7]:
preds = preds[['unique_id', 'ds', 'y_hat']].rename(columns={'y_hat': 'fforma'})

In [10]:
calc_errors_widing(preds, y_test_df, y_insample_train_df, df_seasonality, 'y_hat_naive2').mean()

fforma    0.892159
dtype: float64